In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
os.chdir(r"C:\Users\nites\OneDrive\Desktop\data_sets") 

In [3]:
os.listdir()

['Cambridge Crime Data 2009-2016.csv',
 'Churn_Modelling.csv',
 'deep_learning',
 'hier',
 'HR_comma_sep.csv',
 'li reg',
 'Market Positioning of Mobile',
 'ML with Purushottam Sir.pdf',
 'other_data_set',
 'time_series',
 'yellow_tripdata_2022-09.parquet']

In [4]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.drop(columns=["RowNumber", "Surname"],inplace=True)

In [6]:
df.set_index(["CustomerId"], inplace=True)

In [7]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
CustomerId,,,,,,,,,,,
15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
def credit(x):
    if x<722:
        return "VeryLowCredit"
    elif x >=722 and x<=747:
        return "LowCredit"
    elif x >747 and x <765:
        return "GoodCredit"
    elif x>=765 and x<=777:
        return "VeryGoodCredit"
    else:
        return "ExcellentCredit"

In [9]:
df["CreditScore_cat"]=df["CreditScore"].apply(credit)

In [10]:
# df["Age"].describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9,.99])

In [11]:
def age(x):
    if x<30:
        return "youngAdult"
    elif x>=30 and x<50:
        return "Adult"
    elif x>=50 and x<65:
        return "AeniorAdult"
    else :
        return "Seniors_65+"

In [13]:
df["Age_cat"]=df["Age"].apply(age)

In [14]:
df.dtypes[df.dtypes=="object"].index

Index(['Geography', 'Gender', 'CreditScore_cat', 'Age_cat'], dtype='object')

In [15]:
all_df=pd.get_dummies(df, columns=['Geography', 'Gender', 'CreditScore_cat', 'Age_cat'], drop_first=True)

In [16]:
y= all_df["Exited"]
x=all_df.drop(columns=["Exited"])

In [19]:
!pip install xgboost

  Using cached xgboost-1.7.4-py3-none-win_amd64.whl (89.1 MB)


In [20]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, recall_score, precision_score, precision_recall_curve

In [21]:
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=.3, random_state=88)

In [22]:
dt=DecisionTreeClassifier(min_samples_split=200, min_samples_leaf=20)
dt.fit(x_train, y_train)


DecisionTreeClassifier(min_samples_leaf=20, min_samples_split=200)

In [23]:
print("Train score", dt.score(x_train, y_train))
print("Test score", dt.score(x_test, y_test))

Train score 0.8648571428571429
Test score 0.8553333333333333


In [24]:
params={"max_depth":[7,9,11], 
       "min_samples_leaf":[10,20,30,50,100],
       "min_samples_split":[5,10,20,30],
       "criterion":["gini", "entropy"]}

dt0=DecisionTreeClassifier(random_state=0)
gsearch_tr=GridSearchCV(dt0, param_grid=params, cv=5, n_jobs=-1, verbose=2)
gsearch_tr.fit(x_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=0), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [7, 9, 11],
                         'min_samples_leaf': [10, 20, 30, 50, 100],
                         'min_samples_split': [5, 10, 20, 30]},
             verbose=2)

In [26]:
gsearch_tr.best_params_

{'criterion': 'entropy',
 'max_depth': 7,
 'min_samples_leaf': 20,
 'min_samples_split': 5}

In [27]:
gsearch_tr.best_score_


0.8562857142857142

In [28]:
gsearch_tr.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_leaf=20,
                       min_samples_split=5, random_state=0)

In [29]:
gsearch_tr.classes_

array([0, 1], dtype=int64)

In [30]:
dt1=DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_leaf=20,
                       min_samples_split=5, random_state=0)
dt1.fit(x_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_leaf=20,
                       min_samples_split=5, random_state=0)

In [31]:
print("Train score", dt1.score(x_train, y_train))
print("Test score", dt1.score(x_test, y_test))

Train score 0.8671428571428571
Test score 0.8586666666666667


In [32]:
pred_train=dt1.predict(x_train)
pred_test=dt1.predict(x_test)

In [33]:
pred_train

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [34]:
cm=pd.DataFrame(confusion_matrix(y_train, pred_train),columns=["Pred_NoExisted0", "Pred_Exited1"], index=["Act_NoExited0", "Act_Exited1"])
cm

,Pred_NoExisted0,Pred_Exited1
Act_NoExited0,5401,158
Act_Exited1,772,669


In [35]:
print(classification_report(y_train, pred_train))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92      5559
           1       0.81      0.46      0.59      1441

    accuracy                           0.87      7000
   macro avg       0.84      0.72      0.76      7000
weighted avg       0.86      0.87      0.85      7000



In [36]:
print(classification_report(y_test,pred_test ))

              precision    recall  f1-score   support

           0       0.87      0.96      0.92      2404
           1       0.75      0.43      0.55       596

    accuracy                           0.86      3000
   macro avg       0.81      0.70      0.73      3000
weighted avg       0.85      0.86      0.84      3000



In [37]:
new_pred_train=np.where(pd.DataFrame(dt1.predict_proba(x_train), columns=["prob_0", "prob_1"])["prob_1"] >.8,1,0)
new_pred_train

array([0, 0, 0, ..., 1, 0, 0])

In [38]:
cm=pd.DataFrame(confusion_matrix(y_train, new_pred_train),columns=["Pred_NoExisted0", "Pred_Exited1"], index=["Act_NoExited0", "Act_Exited1"])
cm 

,Pred_NoExisted0,Pred_Exited1
Act_NoExited0,5506,53
Act_Exited1,952,489


In [39]:
print(classification_report(y_train, new_pred_train))

              precision    recall  f1-score   support

           0       0.85      0.99      0.92      5559
           1       0.90      0.34      0.49      1441

    accuracy                           0.86      7000
   macro avg       0.88      0.66      0.70      7000
weighted avg       0.86      0.86      0.83      7000



In [40]:
bg=BaggingClassifier(base_estimator=dt1, n_estimators=300 )
bg.fit(x_train, y_train)

c:\Users\nites\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                        max_depth=7,
                                                        min_samples_leaf=20,
                                                        min_samples_split=5,
                                                        random_state=0),
                  n_estimators=300)

In [41]:
print("Train score", bg.score(x_train, y_train))
print("Test score", bg.score(x_test, y_test))

Train score 0.8677142857142857
Test score 0.857


In [42]:
def eval(model, xtrain, ytrain, xtest, ytest):
    acc=model.score(xtrain, ytrain)
    pred_train=model.predict(xtrain)
    rec=recall_score(ytrain, pred_train)
    prec=precision_score(ytrain, pred_train)
    
    acctest=model.score(xtest, ytest)
    pred_test=model.predict(xtest)
    rec_test=recall_score(ytest, pred_test)
    prec_test=precision_score(ytest, pred_test)
    final={"train_accuracy":acc, "test_accuracy":acctest, "train_recall":rec,"test_recal":rec_test,
           "train_precision":prec,"test_precision":prec_test
          }
    return final

In [43]:
dt_result=pd.DataFrame(eval(dt1, x_train, y_train, x_test, y_test), index=["DecisionTree_hyp"])
dt_result

,train_accuracy,test_accuracy,train_recall,test_recal,train_precision,test_precision
DecisionTree_hyp,0.867143,0.858667,0.464261,0.434564,0.808948,0.748555


In [44]:
bg_result=pd.DataFrame(eval(bg, x_train, y_train, x_test, y_test), index=["Bagging"])
bg_result

,train_accuracy,test_accuracy,train_recall,test_recal,train_precision,test_precision
Bagging,0.867714,0.857,0.483692,0.454698,0.792947,0.722667


In [45]:
final=pd.concat([dt_result, bg_result])
final

,train_accuracy,test_accuracy,train_recall,test_recal,train_precision,test_precision
DecisionTree_hyp,0.867143,0.858667,0.464261,0.434564,0.808948,0.748555
Bagging,0.867714,0.857000,0.483692,0.454698,0.792947,0.722667


## Random forest classifier 

In [46]:
params={"n_estimators":[100,200,300],
       "criterion":["gini", "entropy"],
       "min_samples_split":[10,20,50],
       "min_samples_leaf":[5,10,30],
        "max_features":["sqrt", "log2"]
       }
rf=RandomForestClassifier(random_state=0)
gsearch_rf=GridSearchCV(rf, param_grid=params, cv=5, n_jobs=-1, verbose=2)
gsearch_rf.fit(x_train, y_train)


Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [5, 10, 30],
                         'min_samples_split': [10, 20, 50],
                         'n_estimators': [100, 200, 300]},
             verbose=2)

In [47]:
gsearch_rf.best_params_

{'criterion': 'gini',
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 20,
 'n_estimators': 100}

In [48]:
gsearch_rf.best_estimator_

RandomForestClassifier(min_samples_leaf=5, min_samples_split=20, random_state=0)

In [49]:
rf1=RandomForestClassifier(n_estimators=100,criterion="gini",max_features="sqrt", min_samples_leaf=5, min_samples_split=20, random_state=0)
rf1.fit(x_train, y_train)
rf_result=pd.DataFrame(eval(rf1, x_train, y_train, x_test, y_test), index=["RandomForest_hyp"])
rf_result

,train_accuracy,test_accuracy,train_recall,test_recal,train_precision,test_precision
RandomForest_hyp,0.889714,0.861,0.52533,0.434564,0.895858,0.764012


## Adaboost Classifier

In [50]:
ad=AdaBoostClassifier(base_estimator=rf1,n_estimators=100,learning_rate=.01,)
ad.fit(x_train, y_train)


c:\Users\nites\anaconda3\envs\tf\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostClassifier(base_estimator=RandomForestClassifier(min_samples_leaf=5,
                                                         min_samples_split=20,
                                                         random_state=0),
                   learning_rate=0.01, n_estimators=100)

In [53]:
ad_result=pd.DataFrame(eval(ad, x_train, y_train, x_test, y_test), index=["Adaboost_hyp"])
ad_result.style.highlight_max(color="lightgreen")

,train_accuracy,test_accuracy,train_recall,test_recal,train_precision,test_precision
Adaboost_hyp,0.899571,0.863000,0.565579,0.439597,0.913677,0.772861


## XGBoost Classifier 

In [54]:
from xgboost import XGBClassifier
xg=XGBClassifier()
xg.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [55]:
params={"n_estimators":[100, 200],
       "max_depth":[7,9,11]}
xg1=XGBClassifier(random_state=0)
gsearch_xg=GridSearchCV(xg1, param_grid=params, cv=5, n_jobs=-1,verbose=2)
gsearch_xg.fit(x_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=0, ...),
             n_jobs=-1,
             param_grid={'max_depth': [7, 9, 11], 'n_estimators': [100, 200]},
             verbose=2)

In [56]:
gsearch_xg.best_params_

{'max_depth': 7, 'n_estimators': 100}

In [57]:
from xgboost import XGBClassifier
xg=XGBClassifier(n_estimators=100, max_depth=6)
xg.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [58]:
xg_result=pd.DataFrame(eval(xg, x_train, y_train, x_test, y_test), index=["XGboost_hyp"])

In [59]:
final=pd.concat([xg_result, rf_result, dt_result,ad_result])
final.style.highlight_max(color="pink")

,train_accuracy,test_accuracy,train_recall,test_recal,train_precision,test_precision
XGboost_hyp,0.960571,0.854000,0.827897,0.488255,0.977068,0.686321
RandomForest_hyp,0.889714,0.861000,0.525330,0.434564,0.895858,0.764012
DecisionTree_hyp,0.867143,0.858667,0.464261,0.434564,0.808948,0.748555
Adaboost_hyp,0.899571,0.863000,0.565579,0.439597,0.913677,0.772861
